## Alternative Mutation Strategies
As was discovered in other implementations of the hillclimber research, it appears as if the mutation rate is too slow; for 1000 iterations hillclimber (which equal about 10 hours or computation), the simple hillclimber algorithm with 2-opt seems te be too simple 

In [165]:
from collections import Counter
import requests
import pandas as pd
import geopandas as gpd

import sys

sys.path.append("../")

# from helper_functions import *
# from loading_data import *
from Code.helper_functions import initial_loading, analyze_candidate_solution
from Code.algorithms import random_start_hillclimber, hillclimber

In [87]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()

Do you want to use addresses instead of clusters?True
What stadsdeel do you want to make as a subsection (optional parameter)?T
API data loaded
DB relation POIs loaded
Table all households created
API and DB joined
containers per cluster determined


..\Code\loading_data.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


distance matrix loaded


### Guassian distribution change
This function is a variant of the hillclimber/random algorithm. It randomly selects an amount of containers to swap. This way it balances more or less the exploitation and exploration and makes a more progressive hillclimber. The approach is to set a threshold; the higher the threshold, the less mutation happens to the solution. 

In [197]:
def hillclimber_variable_mutations(df, x=1.9):
    
    df['p'] = np.random.normal(0, 1, size=df.shape[0])
    df_to_change = df[df['p'] > x]
    df = df[df['p'] < x]
    
    rest = df_to_change['rest'].sum() * ['rest']
    plastic = df_to_change['plastic'].sum() * ['plastic']
    papier = df_to_change['papier'].sum() * ['papier']
    glas = df_to_change['glas'].sum() * ['glas']
    textiel = df_to_change['textiel'].sum() * ['textiel']
    fracties = rest + plastic + papier + glas + textiel
    random.shuffle(fracties)

    df_to_change = df_to_change.drop(['rest', 'plastic', 'papier', 'glas', 'textiel'], axis=1)
    print("Amount of clusters to change: " + str(df_to_change.shape[0]))

    fractions_per_cluster = []
    start_point = 0
    for i in range(df_to_change.shape[0]):
        length = df_to_change['totaal'].iloc[i]
        fractions_per_cluster.append(fracties[start_point:start_point +length])
        start_point += length
    df_to_change['new_containers'] = fractions_per_cluster

    df_to_change['rest'], df_to_change['plastic'], df_to_change['papier'], df_to_change['glas'], df_to_change['textiel'] = zip(*df_to_change['new_containers'].apply(lambda x: count(x)))

    df = df.append(df_to_change)
    df = df.drop(['p', 'new_containers'], axis=1)
    return df, df_to_change.shape[0]

def count(lst):
    cnt = Counter()
    for word in lst:
        cnt[word] += 1
    return cnt['rest'], cnt['plastic'], cnt['papier'], cnt['glas'], cnt['textiel']

In [184]:
joined2 = hillclimber_variable_mutations(joined)
joined3 = hillclimber_variable_mutations(joined2)

Amount of clusters to change: 13
Amount of clusters to change: 14


In [179]:
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined2, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined3, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)

Average distance is : 249.6004611676881
Penalties are: 308.1859662117452
Average distance is : 245.76487054832378
Penalties are: 311.91809758572134
Average distance is : 245.18055745386818
Penalties are: 317.10618413433696


In [199]:
def hillclimber2(num_iterations, joined, all_households, rel_poi_df, df_afstandn2, mod_max = 5, parameter='score', complicated=True, clean=True, use_count=False, save=True, prnt=False):
    """
    Function to perform repeated hillclimber. This can be added as a building block
    directly to the standard solution, but also after for example a random algorithm.
    The results are to be seen.
    """
    joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=clean, use_count=use_count)
    method = "Gaussian"

    if parameter == 'score':
        best = avg_distance + penalties
    if parameter == 'penalties':
        best = penalties

    hillclimber_dict = {}
    hillclimber_dict[0] = [avg_distance, penalties, best]

    fractions = ['rest', 'plastic', 'papier', 'glas', 'textiel']
    r = copy.deepcopy(joined)
    for i in range(1, num_iterations+1):
        last = copy.deepcopy(r)

        if method == "2-opt":
            no_modifications = random.randint(1, mod_max)
    #         print(no_modifications)
            for j in range(no_modifications):
                valid = False
                while not valid:
                    location_a = random.randint(0, r.shape[0]-1)
                    fraction_a = random.choice(fractions)
                    location_b = random.randint(0, r.shape[0]-1)
                    fraction_b = random.choice(fractions)

                    if int(r.at[location_a, fraction_b]) > 0 and int(r.at[location_b, fraction_a]) > 0\
                                                        and fraction_a != fraction_b:
                        if prnt:
                            print(r.at[location_a, fraction_a], r.at[location_a, fraction_b], r.at[location_b, fraction_a], r.at[location_b, fraction_b])
                        r.at[location_a, fraction_a] = int(r.at[location_a, fraction_a]) + 1
                        r.at[location_a, fraction_b] = int(r.at[location_a, fraction_b]) - 1
                        r.at[location_b, fraction_a] = int(r.at[location_b, fraction_a]) - 1
                        r.at[location_b, fraction_b] = int(r.at[location_b, fraction_b]) + 1
                        if prnt:
                            print(r.at[location_a, fraction_a], r.at[location_a, fraction_b], r.at[location_b, fraction_a], r.at[location_b, fraction_b])

                        valid = True

        elif method == "Gaussian":
            r, no_modifications = hillclimber_variable_mutations(joined)

        joined_cluster_distance2, good_result_rich2, aansluitingen2, avg_distance2, penalties2 = analyze_candidate_solution(r, all_households, rel_poi_df, df_afstandn2, clean=clean, use_count=use_count)
        hillclimber_dict[i] = [avg_distance2, penalties2, best, no_modifications]
        if parameter == 'score':
            print(avg_distance2+penalties2, best)
            if avg_distance2+penalties2 < best:
                best = avg_distance2+penalties2
            else:
                r = copy.deepcopy(last) # Undo modification
        if parameter == 'penalties':
            print(penalties2, best)
            if penalties2 < best:
                best = penalties2
            else:
                r = copy.deepcopy(last) # Undo modification

    hill_df = pd.DataFrame.from_dict(hillclimber_dict, orient='index')
    hill_df = hill_df.rename(columns={0:'avg_distance', 1:'penalties', 2:'best', 3:'amount of modifications'})

    if save:
        today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)
        hill_df.to_csv('hillclimber' + today + '.csv')
        r.to_csv('hillclimber_best_config' + today + '.csv')

    return hill_df, r


Average distance is : 249.6004611676881
Penalties are: 308.1859662117452
Amount of clusters to change: 13
Average distance is : 250.7009830805045
Penalties are: 310.2849942519229
310.2849942519229 308.1859662117452
Amount of clusters to change: 9
Average distance is : 250.42328986197202
Penalties are: 309.2238427212572
309.2238427212572 308.1859662117452
Amount of clusters to change: 17
Average distance is : 249.80738107249454
Penalties are: 309.35898686931347
309.35898686931347 308.1859662117452


<ipython-input-199-77392ae14401>:69: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)


(   avg_distance   penalties        best  amount of modifications
 0    249.600461  308.185966  308.185966                      NaN
 1    250.700983  310.284994  308.185966                     13.0
 2    250.423290  309.223843  308.185966                      9.0
 3    249.807381  309.358987  308.185966                     17.0,
      cluster_x  cluster_y  s1_afv_rel_nodes_poi  \
 0       124549     481681              379997.0   
 1       124562     481606              111134.0   
 2       124599     481768              142027.0   
 3       124610     481531               93235.0   
 4       124614     481406               68169.0   
 ..         ...        ...                   ...   
 414     128399     479319              249683.0   
 415     129621     479867              271956.0   
 416     129755     479991               15018.0   
 417     129798     479676              412206.0   
 418     129898     479851              160794.0   
 
                                   bk_afv_r